# Funciones auxiliares


In [126]:
import math
import random

def e3_condition(l, u, u_start):
	return l >= (u_start)/4 and l < 3*(u_start)/4 and u >= (u_start)/4 and u < 3*(u_start)/4

def padd_bin(num,m):
	return f"{num:0{m}b}"

def compare_msb(x, y, m):
	x_bin = padd_bin(x,m)
	y_bin = padd_bin(y,m)
	return x_bin[0] == y_bin[0]

def msb(x, m):
	x_bin = padd_bin(x,m)
	return x_bin[0]

def complement_bit(bit):
    return '1' if bit == '0' else '0'

# Función IntegerArithmeticCode

**Definir una función** `IntegerArithmeticCode(mensaje, alfabeto, frecuencias)` **que dado un $mensaje$ en el que se ha utilizado el $alfabeto$ con las correspondientes $frecuencias$ devuelva el resultado de codificar dicho mensaje usando codificación aritmética entera en el intervalo de trabajo $[0,R)$.**


Frecuencias: $f(1)$,..., $f(n)$, $f(i)$ entero

Frecuencias acumuladas:

* $F(0)=0$, 
* $F(i)=\sum_{k=1}^{i}f(k)$,
* $T=F(n)$ suma total de frecuencias


Intervalo de trabajo: $[0,R)$, $R=2^k$, $R>4T$

In [127]:
#Pseudo extraído de "Introduction to Data Compression", Autor: Khalid Sayood, Publisher: Morgan Kaufmann, Año: 2012, Paginas: 113-114
def IntegerArithmeticCode(mensaje, alfabeto, frecuencias):
	code = ''
	T = sum(frecuencias) #Total_count

	#suma de frecuencias
	F = [0] + [sum(frecuencias[:i+1]) for i in range(len(frecuencias))] #Cum_count

	#número k necesitado
	#2^k > 4T => k > log_2(4T) => k > 2 + log_2(T)
	m = math.ceil(2 + math.log2(T))

	#ventana inicial
	l = 0
	u = u_start = pow(2,m) - 1

	#número de escalados del tipo 3
	scale3 = 0

	i = 0
	while i < len(mensaje):
		#tomamos simbolo
		ind = alfabeto.index(mensaje[i])
		
		old_u = u
		old_l = l
		#update u and l
		u = l + math.floor(((old_u - old_l + 1)*F[ind + 1])/(T)) - 1
		l = l + math.floor(((old_u - old_l + 1)*F[ind])/(T))

		while compare_msb(l,u,m) or e3_condition(l,u,u_start + 1):
			if compare_msb(l,u,m):
				#send msb
				b = msb(l,m)
				code += b
				#shift l to the left by 1 bit and shift 0 into LSB
				l = (l << 1) & ~1
				#shift u to the left by 1 bit and shift 1 into LSB
				u = (u << 1) | 1
				#assure values don't go further from 2^m-1
				l = l & u_start
				u = u & u_start
			
				while scale3 > 0:
					#enviar el complemento
					code += complement_bit(b)
					scale3 -= 1

			if e3_condition(l,u,u_start + 1):
				#shift l to the left by 1 bit and shift 0 into LSB
				l = (l << 1) & ~1
				#shift u to the left by 1 bit and shift 1 into LSB
				u = (u << 1) | 1
				#complement (new) MSB of l and u
				mask = 1 << (m - 1)
				l = l ^ mask
				u = u ^ mask
				#assure values don't go further from 2^m-1
				l = l & u_start
				u = u & u_start

				scale3 += 1
		i += 1

	#enviar parte final	
	if l <= T/4:
		code += "01"
		while scale3 > 0:
			code += '1'
			scale3 -= 1
	else:
		code += "10"
		while scale3 > 0:
			code += '0'
			scale3 -= 1

	#añade a
	code += padd_bin(l,m)
	
	return code


# Ejemplo Encode
Atención, el código final no es único.

El proceso para finalizar/cerrar el código ha sido:

* Bits finales pendientes del rescalado: 
Una vez leído todo el mensaje y hecho todos los reescalados posibles el intervalo $[m,M)$ no puede estar contenido en $[0,\frac{R}{2})$, ni en $[\frac{R}{4}, \frac{3R}{4})$, ni en $[\frac{R}{2}, R)$; por lo tanto $[m,M)$ contiene 
$\frac{R}{4}$ y $\frac{R}{2}$, o $\frac{R}{2}$ y $\frac{4R}{4}$. Para asegurarnos que enviamos un número del intervalo en un caso enviamos 01.... y en el otro 10..... 

    
* Envío $m$ representado por exactamente $k$ bits.


In [128]:
alfabeto=['a','b','c','d']
frecuencias=[1,10,20,300]
mensaje='dddcabccacabadac'   
print(alfabeto)
print(frecuencias)
print(mensaje)
print("\n")
C = IntegerArithmeticCode(mensaje,alfabeto,frecuencias)
print(C, len(C))

['a', 'b', 'c', 'd']
[1, 10, 20, 300]
dddcabccacabadac


010001110110000000001000000111111000000100010000000000001100000010001111001100001000000 87


# Ejemplo IntegerArithmeticDecode

**Definir una función** `IntegerArithmeticDecode(codigo, longitud_mensaje, alfabeto, frecuencias)` **que dado un mensaje codificado $codigo$ usando codificación aritmética entera en el intervalo de trabajo $[0,R)$  en el que se ha utilizado el $alfabeto$ con las correspondientes $frecuencias$ devuelva el mensaje original de de longitud $longitud\_mensaje$.**



In [129]:
#Pseudo extraído de "Introduction to Data Compression", Autor: Khalid Sayood, Publisher: Morgan Kaufmann, Año: 2012, Paginas: 117      
def IntegerArithmeticDecode(codigo, longitud_mensaje, alfabeto, frecuencias):
	mensaje = ''
	T = sum(frecuencias) #Total_count

	#suma de frecuencias
	F = [0] + [sum(frecuencias[:i+1]) for i in range(len(frecuencias))] #Cum_count

	#número k necesitado
	#2^k > 4T => k > log_2(4T) => k > 2 + log_2(T)
	m = math.ceil(2 + math.log2(T))

	t_bin = codigo[0:m]
	t = int(t_bin, 2)

	#ventana inicial
	l = 0
	u = u_start = pow(2,m) - 1

	#número de desfase
	offset = 0
	
	while (len(mensaje) < longitud_mensaje):

		#decode symbol x
		freq = math.floor(((t - l + 1) * T - 1) / (u - l + 1))

		for index, freq1 in enumerate(F):
			if freq < freq1:
				sym = alfabeto[index-1]
				ind = index-1
				break

		mensaje += str(sym)
				
		old_u = u
		old_l = l
		#update u and l
		u = l + math.floor(((old_u - old_l + 1)*F[ind + 1])/(T)) - 1
		l = l + math.floor(((old_u - old_l + 1)*F[ind])/(T))

		while compare_msb(l,u,m) or e3_condition(l,u,u_start + 1):
			if compare_msb(l,u,m):
				#shift l to the left by 1 bit and shift 0 into LSB
				l = (l << 1) & ~1
				#shift u to the left by 1 bit and shift 1 into LSB
				u = (u << 1) | 1
				#shift t to the left by 1 bit and read next bit from received bitstream into LSB
				t = (t << 1) | int(codigo[m+offset])
				#assure values don't go further from 2^m-1
				l = l & u_start
				u = u & u_start
				t = int(t & u_start)

				offset += 1

			if e3_condition(l,u,u_start + 1):
				#shift l to the left by 1 bit and shift 0 into LSB
				l = (l << 1) & ~1
				#shift u to the left by 1 bit and shift 1 into LSB
				u = (u << 1) | 1
				#shift t to the left by 1 bit and read next bit from received bitstream into LSB
				t = (t << 1) | int(codigo[m+offset])

				#complement (new) MSB of t, l and u
				mask = 1 << (m - 1)
				t = t ^ mask
				l = l ^ mask
				u = u ^ mask
				#assure values don't go further from 2^m-1
				l = l & u_start
				u = u & u_start
				t = int(t & u_start)

				offset += 1
		
	return mensaje[:longitud_mensaje]

# Ejemplo Decode

In [130]:
alfabeto=['a','b','c','d']
frecuencias=[1,10,20,300]
mensaje='dddcabccacabadac'   
   
print(alfabeto)
print(frecuencias)
print('Tamaño del mensaje:',len(mensaje))
C = IntegerArithmeticCode(mensaje,alfabeto,frecuencias)
print(C, len(C),len(mensaje))

['a', 'b', 'c', 'd']
[1, 10, 20, 300]
Tamaño del mensaje: 16
010001110110000000001000000111111000000100010000000000001100000010001111001100001000000 87 16


In [131]:
mensaje_recuperado=IntegerArithmeticDecode(C,len(mensaje),alfabeto,frecuencias)
print("mensaje: " + mensaje_recuperado)
print(mensaje==mensaje_recuperado)

mensaje: dddcabccacabadac
True


In [132]:
alfabeto=[' ', ',', '.', ';', 'A', 'D', 'F', 'G', 'H', 'I', 'M', 'P', 'S', 'T', 'W', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'y', 'z']
frecuencias=[126, 8, 15, 1, 1, 1, 1, 1, 1, 7, 1, 1, 1, 3, 1, 1, 1, 41, 11, 4, 22, 56, 14, 10, 30, 34, 4, 24, 18, 35, 38, 7, 1, 25, 31, 46, 16, 5, 9, 11, 2]
C='01000000000100110100101011010010000100000011100001110111110100000000111011110110101110000001010100000001011110000100001011011100111100011011110001100001001000110101100111100111111111001111000110010101101001001010101010001100111110101101001011111011010101000100101101011111011111010000001000111110110101111100010011001001110111011000010100110001100001011100011011101001000010010000110010101011001011101001111010111010111010110111101011000100000101001011100101101000111001100011010111100111101101110011110011100011011110111100010101101101011011000101101011101101101000011011000001101110011101001010001000001110111001000011110111100110001101111010000101110110010001011111110000100010010101100000111001101010110001001100100111100100000110101100110110000101110011010110010110010000101101011110011101000010111110110001001000101111110110011111000101101000110101111110010011111100101101100101110011100010001101010111111000011011001000000100111011010111111100100001011010100110011000110011001100111011100110101101001100101111011011100100101010000001100110111101011100001100011010111110101011111101011101010001010111000010010001001000100100011100111010001010101010111110010000010111011111001100011011010011011010010010100110110001101001000001010110111101100101100110011000110001001101100001100011010001001101010100001101011010101001110000000011111101100010110110010010110101111110100100011010010101000111111001010110111011100000110010010100010011000110110001100100111000011100010011011100100001010110010110100001111110011101000001010010011010010111110001101010110110110110010001110010010100110111010101111001110010011010100001011000001011111110001011010110101010111101101001001011100111010011110101100101110001000100110100000010011111100110010100011110010001001010010101000100000101010110111010100111110010100100001000001110101010100100101111011111010100000011011001111010111110001001111100101101001100010010011010011101011100000010101101010001100000100101001101011010101110011111110110000101110111011001100011101111100001011010101101001001100000100101011000010101100100100001000110000111100110011000000100011001011100011100110110000100111110110010001111011111100011101011101111111101001001011101110010001000011101100110111101011000100101001001101001001111100010011000011100001100100011000100000010000010100011011000010010000010000001010100111011011111111110011100010011111001010001101111100001101001111010001000111101010011111100011011110110101001011100100000001000101010011101000101011000001101000100010010101010100100110010111101011110011101001101110100011111001110010011111100100001011000000111010001111010100010010101110110101110101000111100100001100011000001111011011011010101010101111010111001011111010100010010001111010001111011100010110001001101110100000110100101001000111010000111110010110100010110111011100101111100001010110001110011101101101011111110101010000000000101101001011110001011011111000101000000'   
longitud = 665
mensaje_recuperado=IntegerArithmeticDecode(C,longitud,alfabeto,frecuencias)
print(mensaje_recuperado)

The Island of Doctor Moreau, by H. G. Wells. [...] I put down the mouthful that hesitated upon my lips, and listened. Silence, save for the whisper of the morning breeze. I began to think my ears had deceived me. After a long pause I resumed my meal, but with my ears still vigilant. Presently I heard something else, very faint and low. I sat as if frozen in my attitude. Though it was faint and low, it moved me more profoundly than all that I had hitherto heard of the abominations behind the wall. There was no mistake this time in the quality of the dim, broken sounds; no doubt at all of their source. For it was groaning, broken by sobs and gasps of anguish.


In [133]:
alfabeto=[' ', '(', ')', ',', '.', '1', '8', '9', 'A', 'B', 'E', 'G', 'H', 'M', 'S', 'T', 'W', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'y']
frecuencias=[66, 2, 2, 6, 8, 1, 2, 1, 1, 1, 1, 1, 1, 3, 1, 4, 1, 2, 2, 27, 4, 8, 11, 40, 8, 3, 11, 16, 1, 16, 13, 22, 24, 5, 1, 15, 12, 21, 9, 5, 3, 4]
C='010000011110001010101111011011100110011001011101101001101100111110101000010111100011011000111011000101110100111010101110101000001100000010100101110110001011011111010111110110011101010010001010001101001100110010001110010111101111110101010011001111010101010010100001110100011100111110101011111010110100111011001010110010001000110101010101111110111111001010100100011101011100111100010010101101011011101011101000001110111010110110001001110100001010010100101000110111111101011011111011111000010001001100010110111100001000111110011001101110010100011110001001011001010101101001011000101011000011101100001001101000000101100000010101010101010011011110001110100001101110101011111111111011011101111110010100011011101100011010100101100100001000101110110110010100110011010010100000110000011000101100101001110000100110001111010101110101101110101001111111100111010011111000100011001110010110001100110100010111000000101000011001001111011101101111011000100110001000001101110101110001001101111110011010001000111001100001111101001101010101010100110111110010100010000001000110101011000001010011111111110110101010100110011110001101000101000000010110010011011110111011011011010110111110011000100010010100100110000111110000110011101011011110010000011101001010100100001010000000100101110101010100111110101011001010011001100000111110101001010010100110010111100110100101010110110111100010111001100111111100101100010110111010100000110011101011010110000101101100010010000110001011001111101011111001111100111111101000100101111011000101110100101101001000101000111001011100000111101010111111110110111010111101111001000110000111110110011000011110000110000110100001100100011001001101011001011110000000111011011110100011101111111010001010111110110110110011100001010000000'   
longitud = 385
mensaje_recuperado=IntegerArithmeticDecode(C,longitud,alfabeto,frecuencias)
print(mensaje_recuperado)

The Time Machine, H(erbert) G(eorge) Wells [1898] [...] There are balloons. But before the balloons, save for spasmodic jumping and the inequalities of the surface, man had no freedom of vertical movement. Still they could move a little up and down, said the Medical Man. Easier, far easier down than up. And you cannot move at all in Time, you cannot get away from the present moment.


In [134]:
alfabeto = [' ', ',', '.', ';', 'D', 'G', 'H', 'I', 'M', 'T', 'W', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y']
frecuencias = [77, 7, 8, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 22, 4, 6, 13, 35, 10, 7, 19, 24, 1, 20, 7, 16, 24, 11, 19, 16, 34, 12, 3, 8, 1, 8]
mensaje = "The Island of Doctor Moreau, by H. G. Wells. [...] I could not bring myself to approach that black heap. I left it there, with the water rippling round it, under the still stars, and giving it a wide berth pursued my way towards the yellow glow of the house; and presently, with a positive effect of relief, came the pitiful moaning of the puma, the sound that had originally driven me out to explore this mysterious island."
longitud = 424
code=IntegerArithmeticCode(mensaje,alfabeto,frecuencias)
#print(code)
print(code[:1000])
mensaje_recuperado=IntegerArithmeticDecode(code,longitud,alfabeto,frecuencias)
print(mensaje_recuperado==mensaje)

0011110010011110101010100001000100100010110101110110101011101101111011001101100100111111011100101110011010000111001001001110000011011110101001110000101000111110010110010110100100001100101101010111110011010110001101101101111100100101100111001101110100111001100101001111100100110000001001101110010111100001011110110100011100000000111011010111101011001111000001000011111000110100000111110111000010110110001100011111001100001111111011110010010010111011010001001010100110000110010110110000000111100001111010111101101001011010011100001001101111011000011010110001111110100010001101101111000001001000110011011100101011010001101110000000100100011011011111000100001101111100011001010001010111001010100001000101011101100110011011111000000000000000000100101001000110001101111110100011010101100011101101011101001010011001110010100111111100100100110010011100100110000111011001101111001111101101001100011010100010000101011000011101111100101000101110001101101111100000001011010010011100111111101100101001000010001011

In [135]:
alfabeto = [' ', '!', "'", ',', '-', '.', ';', '?', 'B', 'D', 'G', 'H', 'I', 'M', 'O', 'S', 'T', 'W', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y']
frecuencias = [96, 5, 2, 7, 9, 12, 1, 1, 2, 1, 1, 1, 5, 5, 1, 3, 5, 3, 1, 1, 40, 4, 9, 19, 45, 3, 6, 27, 17, 1, 1, 13, 9, 24, 35, 12, 1, 25, 29, 43, 18, 3, 9, 1, 9]
mensaje = "The Island of Doctor Moreau, by H. G. Wells. [...] That way, Mister Blasted Shut-up! that way! roared the captain. Montgomery and his companion turned as he spoke. What do you mean? I said. That way, Mister Blasted Shut-up,--that's what I mean! Overboard, Mister Shut-up,--and sharp! We're cleaning the ship out,--cleaning the whole blessed ship out; and overboard you go! I stared at him dumfounded. Then it occurred to me that it was exactly the thing I wanted. The lost prospect of a journey as sole passenger with this quarrelsome sot was not one to mourn over."
longitud = 565
code=IntegerArithmeticCode(mensaje,alfabeto,frecuencias)
print(code)
print(code[:1000])
mensaje_recuperado=IntegerArithmeticDecode(code,longitud,alfabeto,frecuencias)
print(mensaje_recuperado==mensaje)

0100011000001001100010001000110100011011010001011000001110010100101111100111100100010001111100110101111111100110110010101100101001101010000111001110100111110010011011000011000111101000101000010110011110000110101000110000010000111111011100000111111000111001001110110101110010000000000101001101011011001010010111111101111100100011101001001001100011001010110001100001000011000010010111111111001110101000101100010101000010111011110011111101100010111011010000101010011001110101010100111011110110101011000000011111010100101000010101101001000100101011000010000110110001110111110000110110111100101010111000010010010011010111010011011100110111011000100011101001011010000101110001010111100111100010111000100001110100011001011110111010110001111101101001110110001001111001001001000011010110001100001111111101101110110100111000101001001000111010011110001111111110111111110111100100001100010111110000010011100010011011111101001101101011011100011001001110101110011100101010101001100010110000100011001100110111000001